# Lista de Exercícios 4

Solucionário computacional da Lista de Exercícios 4.

In [2]:
import numpy as np
import sympy as sy

In [3]:
# Funções Implementadas

def gaussjacobi(AB,ER,X0):
    # Função que realiza o cálculo de um sistema linear através do método de iterativo de Gauss-Jacobi.
    #
    # Sinopse:
    #     X = gaussjacobi(AB,ER,X0)
    #
    # Entradas:
    #     AB - Matriz aumentada (np.array) do sistema linear
    #     ER - Erro relativo (forma decimal) entre a iteração i e a iteração i-1
    #     X0 - Vetor estimativa inicial da solução
    # 
    # Saídas:
    #     X - Vetor solução do sistema linear
    
    A = AB[:,0:-1]
    B = AB[:,-1]
    X = X0
    erro = 1
    
    while (erro > ER):
        Xp = np.copy(X)
        for i in range(len(A)):
            s = 0
            for j in range(len(A)):
                if (i != j):
                    s += A[i,j]*Xp[j]
            X[i] = (B[i] - s)/A[i,i]
        EA = np.absolute(X - Xp)
        Emax = np.amax(EA)
        idx = np.where(EA == Emax)
        
        if (len(idx[0]) > 1):
            erro = np.absolute(EA[idx[0][0]]/X[idx[0][0]])
        else:
            erro = np.absolute(EA[idx[0]]/X[idx[0]])
    
    X = np.around(X, decimals=3)

    return X

def gaussseidel(AB,ER,X0):
    # Função que realiza o cálculo de um sistema linear através do método de iterativo de Gauss-Seidel.
    #
    # Sinopse:
    #     X = gaussseidel(AB,ER,X0)
    #
    # Entradas:
    #     AB - Matriz aumentada (np.array) do sistema linear
    #     ER - Erro relativo (forma decimal) entre a iteração i e a iteração i-1
    #     X0 - Vetor estimativa inicial da solução
    # 
    # Saídas:
    #     X - Vetor solução do sistema linear
    
    A = AB[:,0:-1]
    B = AB[:,-1]
    X = X0
    erro = 1
    
    while (erro > ER):
        Xp = np.copy(X)
        for i in range(len(A)):
            s = 0
            for j in range(len(A)):
                if (i != j):
                    s += A[i,j]*X[j]
            X[i] = (B[i] - s)/A[i,i]
        EA = np.absolute(X - Xp)
        Emax = np.amax(EA)
        idx = np.where(EA == Emax)
        
        if (len(idx[0]) > 1):
            erro = np.absolute(EA[idx[0][0]]/X[idx[0][0]])
        else:
            erro = np.absolute(EA[idx[0]]/X[idx[0]])
    
    X = np.around(X, decimals=3)
    
    return X

def jacobian(F,Xs):
    # Função que realiza o cálculo da matriz jacobiana.
    #
    # Sinopse:
    #     J = jacobian(F,Xs)
    #
    # Entradas:
    #     F - Vetor contendo as equações (simbólicas) do sistema não-linear
    #     Xs - Vetor contendo as variáveis (simbólicas) do sistema não-linear
    # 
    # Saídas:
    #      J - Matriz jacobiana (simbólica)
    
    J = sy.zeros(len(Xs), len(Xs))
    
    for i in range(len(Xs)):
        for j in range(len(Xs)):
            J[i,j] = sy.diff(F[i],Xs[j])
    
    return J

def newtonnaolin(F,Xs,ER,X0):
    # Função que realiza o cálculo de um sistema não-linear através do método de iterativo de Newton.
    #
    # Sinopse:
    #     X = newtonnaolin(F,Xs,ER,X0)
    #
    # Entradas:
    #     F  - Vetor (sy.Matrix) contendo as equações (simbólicas) do sistema não-linear
    #     Xs - Vetor (sy.Matrix) contendo as variáveis (simbólico) do sistema não linear
    #     ER - Erro relativo (forma decimal) entre a iteração i e a iteração i-1
    #     X0 - Vetor estimativa inicial da solução
    # 
    # Saídas:
    #     X - Vetor solução do sistema linear
    
    J = jacobian(F,Xs)
    erro = 1
    X = np.copy(X0)
    
    while (erro > ER):
        Xp = np.copy(X)
        A = J[:,:]
        B = F[:,:]
        subs = []
        
        for i in range(len(Xs)):
            subs.append((Xs[i],X[i]))
            
        A = np.asarray(A.subs(subs), dtype=float)
        B = np.asarray(B.subs(subs), dtype=float)
        B *= -1
        
        S = np.linalg.solve(A,B)
        S = np.transpose(S)
        
        X += S[0]
        
        EA = np.absolute(X - Xp)
        Emax = np.amax(EA)
        idx = np.where(EA == Emax)
        
        if (len(idx[0]) > 1):
            erro = np.absolute(EA[idx[0][0]]/X[idx[0][0]])
        else:
            erro = np.absolute(EA[idx[0]]/X[idx[0]])
    
    X = np.around(X, decimals=3)
    
    return X

## Métodos iterativos para sistemas lineares

### Questão 1

Verifique o condicionamento e o critério das linhas para os sistemas abaixo e em seguida aplique o método de Gauss-Jacobi para determinar uma solução aproximada, com erro absoluto inferior a $10^{-2}$, tomando a aproximação inicial $x^{(0)} = 0$.

(a) \begin{cases}
2x_1 − x_2 = 1 \\
x_1 + 2x_2 = 3
\end{cases}

(b) \begin{cases}
x_1 − 0.25x_2 − 0.25x_3 = 0 \\
−0.25x_1 + x_2 − 0.25x_4 = 0 \\
−0.25x_1 + x_3 − 0.25x_4 = 0.25 \\
−0.25x_2 + x_4 = 0.25
\end{cases}

**Obs.** Tome como número de condicionamento o valor $C = det(Ã)$, em que $Ã$ e obtida de $A$ fazendo com que o maior elemento em valor absoluto em cada linha de $Ã$ seja igual a $1$.

In [4]:
# Solução

# (a)
A = np.array([[2., -1.], [1., 2.]]) # Matriz de coeficientes constantes
B = np.array([[1.], [3.]]) # Vetor de constantes
X = np.array([[0.], [0.]]) # Estimativa inicial (X0)
EA = 10**(-2)
erro = 1

while (erro > EA):
    Xp = np.copy(X)
    for i in range(len(A)):
        s = 0
        for j in range(len(A)):
            if (i != j):
                s += A[i,j]*Xp[j]
        X[i] = (B[i] - s)/A[i,i]
    erro = np.amax(np.absolute(X - Xp))

X = np.around(X, decimals=3)    
    
print("(a)\nO número C de condicionamento é:", np.around(np.linalg.det(A), decimals=3))
print("\nA solução do problema é:\n", X)

# (b)
A = np.array([[1., -0.25, -0.25, 0.], [-0.25, 1., 0., -0.25], [-0.25, 0., 1., -0.25], [0., -0.25, 0., 1.]]) # Coeficientes
B = np.array([[0.], [0.], [0.25], [0.25]]) # Vetor de constantes
X = np.array([[0.], [0.], [0.], [0.]]) # Estimativa inicial (X0)
EA = 10**(-2)
erro = 1

while (erro > EA):
    Xp = np.copy(X)
    for i in range(len(A)):
        s = 0
        for j in range(len(A)):
            if (i != j):
                s += A[i,j]*Xp[j]
        X[i] = (B[i] - s)/A[i,i]
    erro = np.amax(np.absolute(X - Xp))

X = np.around(X, decimals=3)    
    
print("\n(b)\nO número C de condicionamento é:", np.around(np.linalg.det(A), decimals=3))
print("\nA solução do problema é:\n", X)

(a)
O número C de condicionamento é: 5.0

A solução do problema é:
 [[0.998]
 [1.002]]

(b)
O número C de condicionamento é: 0.812

A solução do problema é:
 [[0.107]
 [0.093]
 [0.343]
 [0.272]]


### Questão 2

Considere o sistema

\begin{cases}
15c_1 − 2c_2 − c_3 = 3800 \\
−3c_1 + 18c_2 − 6c_3 = 1200 \\
−4c_1 − c_2 + 12c_3 = 2350
\end{cases}

(a) Verifique o condicionamento do sistema.

(b) Verifique o critério das linhas.

(c) Verifique o critério de Sassenfeld.

(d) Determine uma solução aproximada do sistema com erro relativo percentual abaixo de $5\%$ usando o método iterativo de Gauss-Jacobi.

(e) Repita o item (d) usando o método de Gauss-Seidel.

(f) Compare o número de iterações nos itens (d) e (e).

In [5]:
# Solução

AB = np.array([[15., -2., -1., 3800.], [-3., 18., -6., 1200.], [-4., -1., 12., 2350.]])
ER = 5/100
X0 = np.array([[0.], [0.], [0.]])

# (d) Gauss-Jacobi
A = AB[:,0:-1]
B = AB[:,-1]
X = np.copy(X0)
erro = 1
k1 = 0
    
while (erro > ER):
    Xp = np.copy(X)
    for i in range(len(A)):
        s = 0
        for j in range(len(A)):
            if (i != j):
                s += A[i,j]*Xp[j]
        X[i] = (B[i] - s)/A[i,i]
    EA = np.absolute(X - Xp)
    Emax = np.amax(EA)
    idx = np.where(EA == Emax)
    erro = np.absolute(EA[idx[0]]/X[idx[0]])
    k1 += 1

X = np.around(X, decimals=3)    
    
print("(d) A solução de X é:\n", X)

# (e) Gauss-Seidel
A = AB[:,0:-1]
B = AB[:,-1]
X = np.copy(X0)
erro = 1
k2 = 0
    
while (erro > ER):
    Xp = np.copy(X)
    for i in range(len(A)):
        s = 0
        for j in range(len(A)):
            if (i != j):
                s += A[i,j]*X[j]
        X[i] = (B[i] - s)/A[i,i]
    EA = np.absolute(X - Xp)
    Emax = np.amax(EA)
    idx = np.where(EA == Emax)
    erro = np.absolute(EA[idx[0]]/X[idx[0]])
    k2 += 1

X = np.around(X, decimals=3)

print("\n(e) A solução de X é:\n", X)

# (f)
print("\n(f) O número de iterações pelo método de Gauss-Jacobi foi:", k1)
print("O número de iterações pelo método de Gauss-Seidel foi:", k2)

(d) A solução de X é:
 [[301.184]
 [216.637]
 [311.689]]

(e) A solução de X é:
 [[302.072]
 [220.055]
 [314.862]]

(f) O número de iterações pelo método de Gauss-Jacobi foi: 4
O número de iterações pelo método de Gauss-Seidel foi: 3


### Questão 3

Use o método iterativo de Gauss-Seidel para determinar uma solução aproximada do sistema com erro relativo percentual abaixo de $5\%$ dos seguintes sistemas. Antes verifique o critério de Sassenfeld e caso algum dos sitemas não o satisfaça, reorganize o sistema permutando linhas e/ou colunas para garantir a convergência.

(a) \begin{cases}
10x_1 + 2x_2 − x_3 = 27 \\
−3x_1 − 6x_2 + 2x_3 = −61.5 \\
x_1 + x_2 + 5x_3 = −21.5
\end{cases}

(b) \begin{cases}
−3x_1 + x_2 + 12x_3 = 50 \\
6x_1 − x_2 − x_3 = 3 \\
6x_1 + 9x_2 + x_3 = 40
\end{cases}

(c) \begin{cases}
2x_1 − 6x_2 − x_3 = −38 \\
−3x_1 − x_2 + 7x_3 = −34 \\
−8x_1 + x_2 − 2x_3 = −20
\end{cases}

In [6]:
# Solução

# (a)
AB = np.array([[10., 2., -1., 27.], [-3., -6., 2., -61.5], [1., 1., 5., -21.5]])
ER = 5/100
X0 = np.array([0., 0., 0.])
X = gaussseidel(AB,ER,X0)
print("(a)\nA solução do sistema linear é: \n", X)

# (b)
AB = np.array([[6., -1., -1., 3.], [6., 9., 1., 40.], [-3., 1., 12., 50.]])
ER = 5/100
X0 = np.array([0., 0., 0.])
X = gaussseidel(AB,ER,X0)
print("\n(b)\nA solução do sistema linear é: \n", X)

# (c)
AB = np.array([[-8., 1., -2., -20.], [2., -6., -1., -38.], [-3., -1., 7., -34.]])
ER = 5/100
X0 = np.array([0., 0., 0.])
X = gaussseidel(AB,ER,X0)
print("\n(c)\nA solução do sistema linear é: \n", X)

(a)
A solução do sistema linear é: 
 [ 0.5  8.  -6. ]

(b)
A solução do sistema linear é: 
 [1.697 2.829 4.355]

(c)
A solução do sistema linear é: 
 [ 4.005  7.992 -1.999]


### Questão 4
Dos seguintes sistemas abaixo identifique qual(is) deles nao pode(m) ser resolvido(s) usando o método iterativo de Gauss-Seidel.

(a) \begin{cases}
9x + 3y + z = 13 \\
−6x + 8z = 2 \\
2x + 5y − z = 6
\end{cases}

(b) \begin{cases}
x + y + 6z = 8 \\
x + 5y − z = 5 \\
4x + 2y − 2z = 4
\end{cases}

(c) \begin{cases}
−3x + 4y + 5z = 6 \\
−2x + 2y − 3z = −3 \\
2y − z = 1
\end{cases}

#### Solução

(a) Convergente

(b) Divergente 

(c) Divergente

### Questão 5

Uma companhia de eletrônica produz transistores, resistores e chips de computador. Cada transistor usa quatro unidades de cobre, uma unidade de zinco e duas unidades de vidro. Cada resisitor usa três, três e uma unidades de cada material, respectivamente, e cada chip de computador usa duas, uma e três unidades desses materiais, respectivamente. Colocando essas 
informações em uma tabela, tem-se:

Componente | Cobre | Zinco | Vidro
-|-|-|-
Transistor | 4 | 1 | 2
Resistor | 3 | 3 | 1
Chip de Computador | 2 | 1 | 3

O fornecimento desses materiais varia de semana para semana. Assim, a companhia precisa determinar uma meta de produção diferente para cada semana. Por exemplo, em uma semana a quantidade total de materiais disponíveis era 960 unidades de cobre, 510 unidades de zinco e 610 unidades de vidro. Determine o sistema de equações que modela essa meta de produção e determine a sua solução, pelo método de Gauss-Seidel, para determinar o número de transistores, resistores e chips de computador fabricados nessa semana.

In [6]:
# Solução

AB = np.array([[4., 3., 2., 960.], [1., 3., 1., 510.], [2., 1., 3., 610.]])
ER = 1/100
X0 = np.array([[0.], [0.], [0.]])

X = gaussseidel(AB,ER,X0)
X = np.transpose(np.rint(X))
print("[T,R,C] =", X[0])

[T,R,C] = [120. 100.  90.]


## Sistemas não-lineares

### Questão 6

Usando o Método de Newton, determinar uma raiz para cada sistema não-linear abaixo com precisão $\epsilon= 10^{-3}$:

(i) $$\begin{cases} x^2 + y^2 = 2 \\ x^2 - y^2 = 1 \end{cases} \quad 
x^{(0)} = \begin{bmatrix} 1.2 \\ 0.7 \end{bmatrix}$$

(ii) $$\begin{cases} 3x^2 y - y^3 = 4 \\ x^2 - x y^3 = 9 \end{cases} \quad
x^{(0)} = \begin{bmatrix} -1 \\ -2 \end{bmatrix}$$

(i) $$\begin{cases} (x-1)^2 + y^2 = 4 \\ x^2 + (y-1)^2 = 4 \end{cases} \quad
x^{(0)} = \begin{bmatrix} 2 \\ 1 \end{bmatrix}$$

In [7]:
# Solução

x, y = sy.symbols("x, y")
Xs = sy.Matrix([x, y])
ER = 1/100

# (i)
F = sy.Matrix([x**2 + y**2 - 2, x**2 - y**2 - 1])
X0 = np.array([1.2, 0.7])
X = newtonnaolin(F,Xs,ER,X0)
print("(i) A solução do problema é:", X)

# (ii)
F = sy.Matrix([3*(x**2)*y - y**3 - 4, x**2 - x*y**3 - 9])
X0 = np.array([-1., -2.])
X = newtonnaolin(F,Xs,ER,X0)
print("\n(ii) A solução do problema é:", X)

# (iii)
F = sy.Matrix([(x-1)**2 + y**2 - 4, x**2 + (y-1)**2 - 4])
X0 = np.array([2., 1.])
X = newtonnaolin(F,Xs,ER,X0)
print("\n(iii) A solução do problema é:", X)

(i) A solução do problema é: [1.225 0.707]

(ii) A solução do problema é: [3.002 0.148]

(iii) A solução do problema é: [1.823 1.823]
